In [1]:
import json
import pandas as pd
import plotly.graph_objects as go
import hashlib

from IPython.display import display
from tqdm import tqdm
from time import sleep
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 100

In [2]:
with open('raw/Disco Elysium.json', 'r') as f:
    data = json.load(f)

In [3]:
actors = data['actors']
items = data['items']
variables = data['variables']
conversations = data['conversations']
syncInfo = data['syncInfo']


In [4]:
def convert_actors(actors):
    fixed_actors = {}
    needed_fields = [
        'Name', 'Description',
        'character_short_name', 'IsNPC',
        'short_description', 'LongDescription',
    ]
    
    for act in actors:
        idx = act['id']
        fixed_actors[idx] = {}
        for field in act['fields']:
            if field['title'] in needed_fields:
                fixed_actors[idx][field['title']] = field['value']
    return fixed_actors

def convert_dialog_entr(conversations: list):
    fixed_dialog = {}
    needed_fields = [
        'Title', 'Dialogue Text',
        'Actor', 'Conversant',
        #'InputId', 'OutputId',
    ]
    for conversation in conversations:
        for line in conversation['dialogueEntries']:
            idx = (line['conversationID'], line['id'])
            fixed_dialog[idx] = {}
            for field in line['fields']:
                if field['title'] in needed_fields:
                    fixed_dialog[idx][field['title']] = field['value']

            fixed_dialog[idx]['outgoingLinks'] = []
            for link in line['outgoingLinks']:
                fixed_dialog[idx]['outgoingLinks'].append(
                    (link['destinationConversationID'], link['destinationDialogueID']))
            fixed_dialog[idx]['isGroup'] = line['isGroup']
            fixed_dialog[idx]['canvasRect_x'] = line['canvasRect']['x']
            fixed_dialog[idx]['canvasRect_y'] = line['canvasRect']['y']
            fixed_dialog[idx]['canvasRect_width'] = line['canvasRect']['width']
            fixed_dialog[idx]['canvasRect_height'] = line['canvasRect']['height']
    return fixed_dialog
        

In [5]:
actor_table = pd.DataFrame(convert_actors(actors)).T
dia_table = pd.DataFrame(convert_dialog_entr(conversations)).T

In [6]:
dia_table[dia_table['Dialogue Text'].isna() == False]['Dialogue Text'].apply(lambda x: len(x.split(' '))).sum()

1015669

In [7]:
actor_table.to_csv('data_tables/actor.csv')

In [8]:
dia_table.to_csv('data_tables/convs.csv')

# extract all possible dialogues

In [9]:
all_start_nodes = list(dia_table[(dia_table['Title'] == 'START')].index)
links = dia_table['outgoingLinks'].to_dict()

In [10]:
def test_2(d2):
    d2[(0, 0)] = (2, 2)
    return d2

def test_1(d1):
    return test_2(d1)

a_list = []
a = {(0, 0): (0, 0)} 
a_list.append(a)
a_list.append(test_1(a))
a_list

[{(0, 0): (2, 2)}, {(0, 0): (2, 2)}]

In [11]:
def add_one_node(sett):
    path, cur_node = sett
    ret = []
    if len(links[cur_node]) == 0:
        return [], path
    for i in links[cur_node]:
        if i not in path:
            path_c = path.copy()
            path_c[cur_node] = i
            ret.append((path_c, i))
    return ret, None

In [12]:
going_path_list = [({(11, 0): None}, (11, 0)), ({(12, 0): None}, (12, 0))]
#going_path_list = [({(10, 0): None}, (10, 0))]
constr = set([i[1][0] for i in going_path_list])
final_path_list = []

counter = 0
with tqdm(desc=f'Convs: {constr}') as bar:
    
    while len(going_path_list) > 0:
        counter += 1
        if counter % 250000 == 0:
            print(counter, len(going_path_list))
            
        cur_path = going_path_list.pop(0)
        path_to_add, final_path = add_one_node(cur_path)
        going_path_list.extend(path_to_add)
        if final_path is not None:
            final_path_list.append(final_path)
        bar.update(1)
print(len(final_path_list) == 78106, len(final_path_list))

Convs: {11, 12}: 267707it [00:01, 127639.79it/s]

250000 37065


Convs: {11, 12}: 509093it [00:03, 98436.75it/s] 

500000 63450


Convs: {11, 12}: 607070it [00:05, 114602.23it/s]


KeyboardInterrupt: 

In [17]:
from multiprocessing import Lock, Process, Queue, Pool

def do_job(going_path_list):
    print(len(going_path_list))
    final_path_list = []
    while len(going_path_list) > 0:
        cur_path = going_path_list.pop()
        path_to_add, final_path = add_one_node(cur_path)
        going_path_list.extend(path_to_add)
        if final_path is not None:
            final_path_list.append(final_path)
    return final_path_list

In [38]:
#going_path_list = [({(11, 0): None}, (11, 0)), ({(12, 0): None}, (12, 0))]
going_path_list = [({(10, 0): None}, (10, 0))]
constr = set([i[1][0] for i in going_path_list])
final_path_list = []

counter = 0
with tqdm(desc=f'Convs: {constr}') as bar:
    
    while len(going_path_list) > 0:
        counter += 1
        if counter % 250000 == 0:
            break
            
        cur_path = going_path_list.pop(0)
        path_to_add, final_path = add_one_node(cur_path)
        going_path_list.extend(path_to_add)
        if final_path is not None:
            final_path_list.append(final_path)
        bar.update(1)
#print(len(final_path_list) == 78106, len(final_path_list))

Convs: {10}: 0it [00:00, ?it/s]

Convs: {10}: 249999it [00:03, 73073.78it/s]


In [40]:
pr = 8
args = []
for i in range(pr):
    args.append(going_path_list[i::pr])

In [41]:
pr = 8
args = []
for i in range(pr):
    args.append(going_path_list[i::pr])
with Pool(pr) as p:
    
    answer = p.map(do_job, args)

12743
12743
12743
12743
12742
12742
12742
12742


In [42]:
sum([len(i) for i in answer]) + len(final_path_list)

60568

In [ ]:
str_final_path_list = []
for i in final_path_list:
    str_final_path_list.append(str(i))

with open('tmp.json', 'w') as f:
    json.dump(str_final_path_list, f, indent=2)

max([len(i) for i in going_path_list])

# Plotting stuff

In [13]:
idx = 39
canva = dia_table.loc[idx].loc[:, ['canvasRect_x', 'canvasRect_y', 'outgoingLinks', 'Dialogue Text']]

In [20]:
def plot_conv(dia_table, inp_conv_id):
    #idx = 100
    canva = dia_table.loc[inp_conv_id].loc[:, ['canvasRect_x', 'canvasRect_y', 'outgoingLinks', 'Dialogue Text']]

    colors = ['#872341', '#2d98b5']
    dot_color = '#BE3144'
    bg_color = '#22092C'

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=canva['canvasRect_x'],
        y=canva['canvasRect_y'],
        text=canva.index,#['Dialogue Text'],
        mode="markers",
        marker=dict(color=dot_color, )#size=2)
    ))

    for self_dia_id, other in canva.iterrows():
        
        x0 = other['canvasRect_x']
        y0 = other['canvasRect_y']
        links = other['outgoingLinks']
        for conv_id, dia_id in links:
            if conv_id != inp_conv_id:
                continue

            x2, y2 = canva.loc[dia_id][['canvasRect_x', 'canvasRect_y']]
            x1, y1 = (x0 + x2)/2, (y0 + y2)/2
            
            fig.add_trace(go.Scatter(
                x=[x0, x1], y=[y0, y1], showlegend=False, hoverinfo='none',
                mode="lines", line=dict(width=0.5, color=colors[0])))
            
            fig.add_trace(go.Scatter(
                x=[x1, x2], y=[y1, y2], showlegend=False,  hoverinfo='none',
                mode="lines", line=dict(width=0.5, color=colors[1])))

    fig.update_layout(
        autosize=False,
        #xaxis=dict(visible=False),
        #yaxis=dict(visible=False),
        #template='plotly_dark',
        width=1500,
        height=1000,
        margin=dict(l=0, r=0, b=0, t=0, pad=0),
        plot_bgcolor=bg_color,
        paper_bgcolor=bg_color,
    )

    fig.update_shapes(dict(xref='x', yref='y'))
    return fig

In [21]:
def plot_3d_conv(dia_table, from_idx, to_idx):
    #idx = 100
    canva = dia_table.loc[from_idx:to_idx].loc[:, ['canvasRect_x', 'canvasRect_y', 'outgoingLinks', 'Dialogue Text']]

    colors = ['#872341', '#2d98b5']
    dot_color = '#BE3144'
    bg_color = '#22092C'

    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=canva['canvasRect_x'],
        y=canva['canvasRect_y'],
        z=canva.index.get_level_values(0),
        text=canva['Dialogue Text'],
        mode="markers",
        marker=dict(color=dot_color, size=2)
    ))

    for (self_conv_id, self_dia_id), other in canva.iterrows():
        
        x0 = other['canvasRect_x']
        y0 = other['canvasRect_y']
        z0 = self_conv_id
        links = other['outgoingLinks']
        for conv_id, dia_id in links:
            if conv_id not in canva.index:
                continue
            z2 = conv_id
            x2, y2 = canva.loc[conv_id, dia_id][['canvasRect_x', 'canvasRect_y']]
            
            x1, y1, z1 = (x0 + x2)/2, (y0 + y2)/2, (z0 + z2)/2
            fig.add_trace(go.Scatter3d(
                x=[x0, x1], y=[y0, y1], z=[z0, z1], showlegend=False, hoverinfo='none',
                mode="lines", line=dict(width=0.5, color=colors[0])))
            
            fig.add_trace(go.Scatter3d(
                x=[x1, x2], y=[y1, y2], z=[z1, z2],showlegend=False,  hoverinfo='none',
                mode="lines", line=dict(width=0.5, color=colors[1])))

    fig.update_layout(
        autosize=False,
        #xaxis=dict(visible=False),
        #yaxis=dict(visible=False),
        #template='plotly_dark',
        width=1500,
        height=1000,
        margin=dict(l=0, r=0, b=0, t=0, pad=0),
        plot_bgcolor=bg_color,
        paper_bgcolor=bg_color,
    )

    fig.update_shapes(dict(xref='x', yref='y'))
    return fig

In [22]:
fig = plot_3d_conv(1, 14)
fig.show()

TypeError: plot_3d_conv() missing 1 required positional argument: 'to_idx'

In [23]:
fig = plot_conv(dia_table, idx)
fig.show()

In [17]:
dia_table

Title outgoingLinks isGroup canvasRect_x canvasRect_y  \
1    0   START      [(1, 1)]       0          0.0          0.0   
     1   input            []       1          0.0          0.0   
2    0   START      [(2, 1)]       0          0.0          0.0   
     1   input            []       1          0.0          0.0   
3    0   START      [(3, 1)]       0          0.0          0.0   
...        ...           ...     ...          ...          ...   
1499 52         [(1499, 20)]       0     4413.846   -12.307692   
     53         [(1499, 12)]       0   4396.92334  -1085.38464   
     54         [(1499, 36)]       0   5051.53857   247.692322   
1500 0   START            []       0          0.0          0.0   
1501 0   START            []       0          0.0          0.0   

        canvasRect_width canvasRect_height Actor Conversant  \
1    0             160.0              30.0   NaN        NaN   
     1             160.0              30.0     0        NaN   
2    0             160.0              30.0   NaN        NaN   
     1             160.0              30.0     0        NaN   
3    0             160.0              30.0   NaN        NaN   
...                  ...               ...   ...        ...   
1499 52              0.0               0.0   396          0   
     53              0.0               0.0   396          0   
     54              0.0               0.0   408          0   
1500 0             160.0              30.0   NaN        NaN   
1501 0             160.0              30.0   NaN        NaN   

                                               Dialogue Text  
1    0                                                   NaN  
     1                                                   NaN  
2    0                                                   NaN  
     1                                                   NaN  
3    0                                                   NaN  
...                                                      ...  
1499 52               "I'm... not ready to go to sleep yet."  
     53  "It is I, Garte, Manager of this Faire Cafeterium."  
     54               Sleep is what he needs most right now.  
1500 0                                                   NaN  
1501 0                                                   NaN  

[112962 rows x 10 columns]